Autoencoder for Brain MRI Classification with Transfer Learning

In [17]:
#Necessary imports
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

Load and Preprocess Image Data

<div style="font-size:85%">
  
1. Set the folder path for images.  
2. Read the labels CSV file and clean filenames.  
3. Initialize lists for images and labels.  
4. Loop over each row:  
   1. Get filename and label.  
   2. Load image.  
   3. Resize to 64x64.  
   4. Append to lists.  
5. Convert to NumPy arrays.  
6. Print shapes. 

In [18]:
folder_path = "C:/Users/rithv/GitHub/brain-mri-scans/code/images/"

df = pd.read_csv("labels.csv")
df["filename"] = df["filename"].astype(str).str.strip()

images = []
labels = []

for i, row in df.iterrows():
    filename = row["filename"]
    label = row["target"]

    img_path = os.path.join(folder_path, filename)
    img = cv2.imread(img_path)

    if img is None:
        print("Skipping:", img_path)
        continue

    img = cv2.resize(img, (64, 64))
    images.append(img)
    labels.append(float(label))

x = np.array(images)   # <-- REAL NumPy array again
y = np.array(labels)

print("x:", x.shape, type(x))
print("y:", y.shape, type(y))


x: (300, 64, 64, 3) <class 'numpy.ndarray'>
y: (300,) <class 'numpy.ndarray'>


Convert Images to Grayscale and Print Shape

In [19]:
x_gray = np.mean(x, axis=-1, keepdims=True)   # (300, 64, 64, 1)
print(x_gray.shape)

(300, 64, 64, 1)


Augment Data for Training

<div style="font-size:85%">
  
1. Create an ImageDataGenerator for augmentation.  
2. Generate augmented images to reach a count of 3000.  
3. Convert to NumPy array.  
4. Print shape.  

</div>

In [20]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

augmented_img = []
count = 3000

for batch in datagen.flow(x_gray, batch_size=32, shuffle=False):
    for img in batch:
        augmented_img.append(img)
        if len(augmented_img) >= count:
            break
    if len(augmented_img) >= count:
        break

x_aug = np.array(augmented_img)
print(x_aug.shape)   # (3000, 64, 64, 1)

(3000, 64, 64, 1)


Normalize Data and Perform Train Test Split

<div style="font-size:85%">
  
1. Normalize grayscale images to [0,1].  
2. Split x_gray and y into training and testing sets with 80/20 ratio.  

</div>

In [21]:
x_gray = x_gray / 255.0
x_aug = x_aug / 255.0

x_train, x_test, y_train, y_test = train_test_split(x_gray, y, test_size=0.2, random_state=42)


Define Autoencoder Building Function

<div style="font-size:85%">
  
1. Define a function to build the autoencoder and encoder models.  
2. Create input layer.  
3. Build encoder with Conv2D and MaxPooling2D.  
4. Build decoder with Conv2D and UpSampling2D.  
5. Create autoencoder and encoder models.  
6. Return both models.  

</div>

In [22]:
def build_autoencoder(input_shape=(64, 64, 1)):
    input_img = Input(shape=input_shape)

    # Encoder
    z = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    z = MaxPooling2D((2, 2), padding='same')(z)

    z = Conv2D(64, (3, 3), activation='relu', padding='same')(z)
    encoded = MaxPooling2D((2, 2), padding='same')(z)

    # Decoder
    z = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    z = UpSampling2D((2, 2))(z)

    z = Conv2D(32, (3, 3), activation='relu', padding='same')(z)
    z = UpSampling2D((2, 2))(z)

    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(z)

    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, encoded)

    return autoencoder, encoder

Define Autoencoder Training Function

<div style="font-size:85%">
  
1. Define a function to compile and train the autoencoder.  
2. Compile with Adam optimizer and MSE loss.  
3. Fit on augmented data with validation split.  
4. Return the trained autoencoder.  

</div>

In [23]:
def train_autoencoder(autoencoder, x_aug, epochs=20, batch_size=32):
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(x_aug, x_aug, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    return autoencoder



Define Classifier Building Function for Transfer Learning

<div style="font-size:85%">
  
1. Define a function to build the classifier using the encoder.  
2. Freeze encoder layers.  
3. Create Sequential model with encoder, Flatten, Dense layers.  
4. Return the classifier.  

</div>

In [24]:
def build_classifier(encoder):
    for layer in encoder.layers:
        layer.trainable = False

    classifier = Sequential([
        encoder,
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return classifier

Define Classifier Training Function

<div style="font-size:85%">
  
1. Define a function to compile and train the classifier.  
2. Compile with Adam optimizer, binary_crossentropy loss, and accuracy metric.  
3. Fit on training data with validation split.  
4. Return the trained classifier.  

</div>

In [25]:
def train_classifier(classifier, x_train, y_train, epochs=10, batch_size=32):
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    return classifier

Define Classifier Evaluation Function

<div style="font-size:85%">
  
1. Define a function to evaluate the classifier on test data.  
2. Evaluate and return accuracy.  

</div>

In [26]:
def evaluate_classifier(classifier, x_test, y_test):
    loss, accuracy = classifier.evaluate(x_test, y_test)
    return accuracy

In [27]:
#Build and train autoencoder
autoencoder, encoder = build_autoencoder()
autoencoder = train_autoencoder(autoencoder, x_aug)

# Build and train classifier
classifier = build_classifier(encoder)
classifier = train_classifier(classifier, x_train, y_train)

# Evaluate classifier
test_accuracy = evaluate_classifier(classifier, x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 23s 199ms/step - loss: 0.0370 - val_loss: 0.0247
Epoch 2/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 198ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 3/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 195ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 4/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 195ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 5/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 197ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 6/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 21s 199ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 7/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 197ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 8/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 201ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 9/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 202ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 10/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 195ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 11/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 196ms/step - loss: 0.0246 - val_loss: 0.0247
Epoch 12/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 20